This code imports the necessary libraries (pandas, numpy, matplotlib, and Path from the pathlib module) to handle data analysis and plotting. It then searches for the accelerometer data file 0_z.csv in two possible locations and selects the one that exists. The CSV file is read into a pandas DataFrame, ignoring any lines starting with # (used for comments). 

- The column names are cleaned by stripping spaces and converting them to lowercase. The main data columns (t, x, y, and z) are converted to numeric values, forcing any invalid entries to become missing values (NaN). 
- Next, any rows containing missing data are removed, and the dataset is sorted by the time column (t) before resetting the index to keep it tidy. 

- Finally, the code displays the first few rows of the cleaned dataset to confirm that the data has been properly loaded and prepared for further analysis.

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

csv_candidates = [Path('0_z.csv'), Path('/mnt/data/0_z.csv')]
for p in csv_candidates:
    if p.exists():
        csv_path = p
        break

df = pd.read_csv(csv_path, comment='#')
df.columns = [c.strip().lower() for c in df.columns]
for c in ['t','x','y','z']:
    df[c] = pd.to_numeric(df[c], errors='coerce')
df = df.dropna(subset=['t','x','y','z']).sort_values('t').reset_index(drop=True)
df.head()

,t,x,y,z
0,0.00,-0.0938,-0.0156,0.9531
1,0.02,-0.0938,-0.0156,0.9531
2,0.04,-0.0938,-0.0156,0.9531
3,0.06,-0.0938,-0.0156,0.9531
4,0.08,-0.0938,-0.0156,0.9531
